Importing packages and modules

In [21]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import json
import pandas as pd
from datetime import datetime,time,timedelta
import pickle

# Scraping the tables for data
First step is to gather all the URLs for the different seasons and competitions.

In [22]:
base_url = 'https://understat.com/league'
leagues = ['EPL','La_liga','Bundesliga','Serie_A','Ligue_1','RFPL'] 
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

# The entirety of this code can be looped over either all leagues or all seasons (or both !) for more data 

Retrieving the data from the URLs

In [23]:
url = base_url + '/' + leagues[0] + '/' + seasons[7]
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

# finding the appropriate scripts
scripts = soup.find_all('script')

json_string = ''

for el in scripts:
    if 'teamsData' in str(el):
        json_string = str(el).strip()

# finding the start and end point of the data
json_string_start = json_string.index("('") + 2
json_string_end = json_string.index("')")

json_data = json_string[json_string_start:json_string_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

## Putting data into pandas dataframes
Now that we have extracted the json data we can convert it into a python dictionary

In [24]:
#loading in the json data
data = json.loads(json_data)
teams = {}
#get teams and their relevant ids and put them into seperate dictionary
for id in data.keys():
    teams[id] = data[id]['title']
print(teams)


{'71': 'Aston Villa', '72': 'Everton', '74': 'Southampton', '75': 'Leicester', '78': 'Crystal Palace', '79': 'Norwich', '80': 'Chelsea', '81': 'West Ham', '82': 'Tottenham', '83': 'Arsenal', '86': 'Newcastle United', '87': 'Liverpool', '88': 'Manchester City', '89': 'Manchester United', '90': 'Watford', '92': 'Burnley', '220': 'Brighton', '229': 'Wolverhampton Wanderers', '244': 'Brentford', '245': 'Leeds'}


In [25]:
#list of column names
columns = []
#list of values from first datapoint
values = []
#taking the column headers and values of said columns from the the array of 'history' dictionaries for the first game for each team
for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break
#taking the value here can be used to verify the voracity of the data you collect.

In [26]:
#getting data for all the teams
list_of_dataframes_h = {}
list_of_dataframes_a = {}
for id, team in teams.items():
    teams_data = []
    for row in data[id]['history']:
        teams_data.append(list(row.values()))
    
    df = pd.DataFrame(teams_data,columns = columns)
    #this section of code is used to split data into home and away tables
    list_of_dataframes_h[team] = df.loc[df['h_a'] == 'h']
    list_of_dataframes_a[team] = df.loc[df['h_a'] == 'a']
    print(f"Added data for {team}")
    
    

Added data for Aston Villa
Added data for Everton
Added data for Southampton
Added data for Leicester
Added data for Crystal Palace
Added data for Norwich
Added data for Chelsea
Added data for West Ham
Added data for Tottenham
Added data for Arsenal
Added data for Newcastle United
Added data for Liverpool
Added data for Manchester City
Added data for Manchester United
Added data for Watford
Added data for Burnley
Added data for Brighton
Added data for Wolverhampton Wanderers
Added data for Brentford
Added data for Leeds


### Formatting the data
Now we quickly format the data into a more friendly format

In [27]:
for team,df in list_of_dataframes_h.items():
    list_of_dataframes_h[team]['ppda_coef'] = list_of_dataframes_h[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    list_of_dataframes_h[team]['oppda_coef'] = list_of_dataframes_h[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    
for team,df in list_of_dataframes_a.items():
    list_of_dataframes_a[team]['ppda_coef'] = list_of_dataframes_a[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    list_of_dataframes_a[team]['oppda_coef'] = list_of_dataframes_a[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

Now we can sum and take the average of the metrics to have a cumulative table for the entire season.

In [28]:
columns_to_sum = cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
columns_to_avg = ['ppda_coef', 'oppda_coef']

In [29]:
frames_h = []
frames_a = []
for team, df in list_of_dataframes_h.items():
  sum_data = pd.DataFrame(df[columns_to_sum].sum()).transpose()
  avg_data = pd.DataFrame(df[columns_to_avg].mean()).transpose()
  final_df_h = sum_data.join(avg_data)
  final_df_h['team'] = team
  final_df_h['matches'] = len(df)
  frames_h.append(final_df_h)

for team, df in list_of_dataframes_a.items():
  sum_data = pd.DataFrame(df[columns_to_sum].sum()).transpose()
  avg_data = pd.DataFrame(df[columns_to_avg].mean()).transpose()
  final_df_a = sum_data.join(avg_data)
  final_df_a['team'] = team
  final_df_a['matches'] = len(df)
  frames_a.append(final_df_a)
  
full_stat_h = pd.concat(frames_h)
full_stat_a = pd.concat(frames_a)
full_stat_a.head()

,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef,team,matches
0,10.60,15.35,9.84,12.96,79.00,74.00,10.00,16.00,10.97,3.00,0.00,7.00,9.00,-3.12,13.51,8.48,Aston Villa,10
0,10.86,15.30,9.34,14.54,44.00,79.00,9.00,18.00,11.40,1.00,3.00,6.00,6.00,-5.20,18.16,8.26,Everton,10
0,13.46,20.37,11.18,19.61,62.00,75.00,12.00,24.00,10.83,2.00,3.00,6.00,9.00,-8.43,10.72,10.14,Southampton,11
0,10.85,17.09,10.85,14.05,53.00,67.00,15.00,21.00,7.88,2.00,3.00,4.00,9.00,-3.20,14.39,11.34,Leicester,9
0,7.85,16.70,7.85,15.18,47.00,104.00,10.00,19.00,7.99,1.00,4.00,5.00,7.00,-7.33,22.23,8.07,Crystal Palace,10


Re-ordering columns for better readability

In [30]:
full_stat_h = full_stat_h[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat_h.sort_values('pts', ascending=False, inplace=True)
full_stat_h.reset_index(inplace=True, drop=True)
full_stat_h['position'] = range(1,len(full_stat_h)+1)

full_stat_a = full_stat_a[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat_a.sort_values('pts', ascending=False, inplace=True)
full_stat_a.reset_index(inplace=True, drop=True)
full_stat_a['position'] = range(1,len(full_stat_a)+1)

full_stat_h.head()

,team,matches,wins,draws,loses,scored,missed,pts,xG,npxG,xGA,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,position
0,Manchester City,11,9.00,1.00,1.00,32.00,6.00,28.00,27.33,25.05,6.55,6.55,18.49,8.57,32.05,144.00,36.00,26.48,1
1,Liverpool,10,7.00,3.00,0.00,25.00,6.00,24.00,26.03,24.51,6.42,6.42,18.09,7.73,23.54,107.00,43.00,24.73,2
2,Tottenham,10,7.00,1.00,2.00,16.00,10.00,22.00,17.64,17.64,12.23,12.23,5.41,13.67,11.87,78.00,83.00,17.63,3
3,Arsenal,10,7.00,1.00,2.00,18.00,8.00,22.00,20.08,17.68,10.57,9.81,7.88,13.44,14.50,89.00,51.00,20.63,4
4,Chelsea,11,5.00,5.00,1.00,25.00,10.00,20.00,22.95,19.91,11.15,9.63,10.28,9.20,19.29,104.00,45.00,22.10,5


Now we add xG difference into the dataframe

In [31]:
full_stat_h['xG_diff'] = full_stat_h['xG'] - full_stat_h['scored']
full_stat_h['xGA_diff'] = full_stat_h['xGA'] - full_stat_h['missed']
full_stat_h['xpts_diff'] = full_stat_h['xpts'] - full_stat_h['pts']

full_stat_a['xG_diff'] = full_stat_a['xG'] - full_stat_a['scored']
full_stat_a['xGA_diff'] = full_stat_a['xGA'] - full_stat_a['missed']
full_stat_a['xpts_diff'] = full_stat_a['xpts'] - full_stat_a['pts']

Now we convert to the appropriate datatypes for each column

In [32]:
columns_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat_h[columns_to_int] = full_stat_h[columns_to_int].astype(int)
full_stat_a[columns_to_int] = full_stat_a[columns_to_int].astype(int)

And now we make the final output cleaner

In [33]:
column_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat_h = full_stat_h[column_order]
full_stat_h.columns = ['#', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xG Diff', 'NPxG', 'xGA', 'xGA Diff', 'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'xPTS diff']
full_stat_a = full_stat_a[column_order]
full_stat_a.columns = ['#', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xG Diff', 'NPxG', 'xGA', 'xGA Diff', 'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'xPTS diff']

pd.options.display.float_format = '{:,.2f}'.format

In [34]:
full_stat_h.head()

,#,Team,M,W,D,L,G,GA,PTS,xG,...,xGA,xGA Diff,NPxGA,NPxGD,PPDA,OPPDA,DC,ODC,xPTS,xPTS diff
0,1,Manchester City,11,9,1,1,32,6,28,27.33,...,6.55,0.55,6.55,18.49,8.57,32.05,144,36,26.48,-1.52
1,2,Liverpool,10,7,3,0,25,6,24,26.03,...,6.42,0.42,6.42,18.09,7.73,23.54,107,43,24.73,0.73
2,3,Tottenham,10,7,1,2,16,10,22,17.64,...,12.23,2.23,12.23,5.41,13.67,11.87,78,83,17.63,-4.37
3,4,Arsenal,10,7,1,2,18,8,22,20.08,...,10.57,2.57,9.81,7.88,13.44,14.50,89,51,20.63,-1.37
4,5,Chelsea,11,5,5,1,25,10,20,22.95,...,11.15,1.15,9.63,10.28,9.20,19.29,104,45,22.10,2.10


In [35]:
full_stat_h.head()

,#,Team,M,W,D,L,G,GA,PTS,xG,...,xGA,xGA Diff,NPxGA,NPxGD,PPDA,OPPDA,DC,ODC,xPTS,xPTS diff
0,1,Manchester City,11,9,1,1,32,6,28,27.33,...,6.55,0.55,6.55,18.49,8.57,32.05,144,36,26.48,-1.52
1,2,Liverpool,10,7,3,0,25,6,24,26.03,...,6.42,0.42,6.42,18.09,7.73,23.54,107,43,24.73,0.73
2,3,Tottenham,10,7,1,2,16,10,22,17.64,...,12.23,2.23,12.23,5.41,13.67,11.87,78,83,17.63,-4.37
3,4,Arsenal,10,7,1,2,18,8,22,20.08,...,10.57,2.57,9.81,7.88,13.44,14.50,89,51,20.63,-1.37
4,5,Chelsea,11,5,5,1,25,10,20,22.95,...,11.15,1.15,9.63,10.28,9.20,19.29,104,45,22.10,2.10


We can save this dataframe to be used for other projects. We can save it either as a csv or use the pickling module to save it. I chose to pickle it due to ease of use for further projects.

In [36]:
full_stat_h.to_pickle("./EPL_table_h.pkl")

full_stat_a.to_pickle("./EPL_table_a.pkl") 

# Scraping Fixtures
You can use a similar method to scrape the fixtures for the next week from understat.

In [37]:
#this time we are looking for 'datesData' in te script
#clearing the json string
json_string = ''

for el in scripts:
    if 'datesData' in str(el):
        json_string = str(el).strip()

# again finding the start and end point of the data
json_string_start = json_string.index("('") + 2
json_string_end = json_string.index("')")

json_data = json_string[json_string_start:json_string_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

data = json.loads(json_data)

Now we have the json fixtures data loaded we notice that it takes the form of a list of dictionaries with every fixture. Since we only want the fixtures for the next week we need to define a range to work in. This is done by finding the end date for this week and searching inbetween now and the end of week date.

In [38]:
#bit of code to find the end of this week
next_week = datetime.combine((datetime.today() + timedelta(days = 7)).date(),time(23,59))
next_week

datetime.datetime(2022, 1, 24, 23, 59)

Now we take all the fixtures between today and and the end of the week and store it into a dictionary and create a dataframe out of them.

In [39]:
fixtures_data = {'Home Team': [],'Away Team': [],'Home Team id': [],'Away Team id': []}
today = datetime.combine(datetime.now().date(),time(0,0))
for fixture in data:
    fixture_date = datetime.strptime(fixture['datetime'],'%Y-%m-%d %H:%M:%S')
    if fixture_date > today and fixture_date < next_week:
        fixtures_data["Home Team"].append(fixture['h']['title'])
        fixtures_data["Away Team"].append(fixture['a']['title'])
        fixtures_data["Home Team id"].append(fixture['h']['id'])
        fixtures_data["Away Team id"].append(fixture['a']['id'])
fixtures_df = pd.DataFrame(fixtures_data)
fixtures_df

,Home Team,Away Team,Home Team id,Away Team id
0,Watford,Norwich,90,79
1,Everton,Aston Villa,72,71
2,Arsenal,Burnley,83,92
3,Brentford,Wolverhampton Wanderers,244,229
4,Leeds,Newcastle United,245,86
5,Manchester United,West Ham,89,81
6,Southampton,Manchester City,74,88
7,Crystal Palace,Liverpool,78,87
8,Leicester,Brighton,75,220
9,Chelsea,Tottenham,80,82


This code can be tweaked to extract more data or to look for historical data. Again, we can dump this data into a .pickle format to make it easy to use in later projects.

In [40]:
fixtures_df.to_pickle("./EPL_fixtures.pkl")